### Haha 2021
### Convertir los datasets de las competencias a json y separarlos 75/25 train/test
- Se crea el modelo evomsa pre-entrenado haha2021_Es.evomsa
- En Single f1_score_macro  = 0.823913749710607
- En Muliple f1_score_macro = 0.8363469487658519
  - exist21_model, davincis22_model, meoffendes21_model


In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [28]:
# Exist2021

df = pd.read_csv("../haha_2021_train.csv", usecols=['text', 'is_humor']) #,   header=0, nrows=50)# , nrows=30, header=1)
df.rename(columns = {'is_humor':'klass'}, inplace = True)

In [29]:
df.shape

(24000, 2)

In [30]:
df

,text,klass
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,1
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1
3,No se porqué me hago la cabeza deooos,0
4,Quisiera saber que hago durante la siesta de l...,0
...,...,...
23995,"—Estoy afuera de tu casa, sal. —Ya voy, pimienta.",1
23996,Un kilo de helado no me vendría mal,0
23997,Soy la única que no le parece tan mierda el 20...,0
23998,Alguien tiene cuenta de Netflix que quiera com...,0


In [31]:
np.unique(df['klass'])
# 1 es humor
# 0 No es humor

array([0, 1], dtype=int64)

In [32]:
# df["klass"] = np.where(df["klass"] == "sexist", 0, 1)

In [33]:
# df

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['klass'], train_size=0.75)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((18000,), (6000,), (18000,), (6000,))

In [36]:
type(X_train)

pandas.core.series.Series

In [37]:
df_train = pd.concat([X_train, y_train], axis=1)
print(df_train.shape)
df_train

(18000, 2)


,text,klass
654,#TengoMiedoA Terminar un buen tweet y que teng...,1
19724,Podrían informarme @ISSSTE_mx por qué aún no s...,0
3062,"Mi madre se queja de mi, pero si supiera lo qu...",0
18484,"Si no exagera, fracasó como madre.",1
19828,—¿Y POR QUÉ NO PUEDO SALIR EN LA FOTO? ¿ES QUE...,1
...,...,...
2669,"#ElFísicoAtraePero que te diga: ""no te vayas, ...",0
4099,Que hago?,0
2001,¿Por qué te caen tan bien los submarinistas?\n...,1
9724,"2am y ando escuchando a GG Allin, me entra la ...",0


In [38]:
df_test = pd.concat([X_test, y_test], axis=1)
print(df_test.shape)
df_test

(6000, 2)


,text,klass
12165,Esos dubbys en plena Walmart NO son de Dios!! ...,0
12586,"- ¡Papá, Papá, llévame al circo!\n- No hijo, q...",1
9927,Vamos Argentina! Quien más quiere que gane la ...,0
13106,Que rica es la tortilla lpm,0
11997,me tatuaria en la frente ese tw,0
...,...,...
2123,La gente notoria me irrita 😐,0
669,"Soy una maravilla de ex, no fastidio, no extra...",0
16158,LEVÁNTENSE! Si no los ha tumbado el hambre no ...,0
19083,🔋100% 😂\n🔋90% 😆 \n🔋80% 😄 \n🔋70% 😋 \n🔋60% 😅 \n🔋...,1


In [39]:
with open('..\haha2021_Es_train.json', 'w') as f:
    f.write(df_train.to_json(orient='records', lines=True))
    
with open('..\haha2021_Es_test.json', 'w') as f:
    f.write(df_test.to_json(orient='records', lines=True))

In [40]:
from EvoMSA.base import EvoMSA
from microtc.utils import save_model
from sklearn import metrics
from microtc.utils import load_model
from microtc.utils import save_model

In [41]:
evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, models=None)

# Pq a .squeeze() ? 

In [42]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))

100%|██████████| 30/30 [00:05<00:00,  5.54it/s]

0.823913749710607


In [45]:
# crear modelo pre-entrenado en Sigle par evitar efecto cascada
save_model(evo, '..\\..\\single_models\\haha21_single_Es.evomsa') 

### Multiple
---

In [49]:
stacked_method = "sklearn.svm.LinearSVC"
# usar = "sklearn.naive_bayes.BernoulliNB"
# usar = "sklearn.naive_bayes.MultinomialNB"

### Descarto haha2021 (y 2018 caduca)

exist21_model = [load_model('../../single_models/exist21_single_Es.evomsa'), stacked_method]
# haha18_model = [load_model('..\\..\\single_models\\haha2018_Es.evomsa'), stacked_method]
#mexa3t_model = [load_model('..\\..\\single_models\\mexa3t2018_aggress_Es.evomsa'), stacked_method]
#misoginia_model = [load_model('..\\..\\single_models\\misoginia_Es.evomsa'), stacked_method]
davincis22_model = [load_model('../../single_models/davincis22_single_Es.evomsa'), stacked_method]
meoffendes21_model = [load_model('../../single_models/meoffendes21_single_Es.evomsa'), stacked_method]

mm = [exist21_model, davincis22_model, meoffendes21_model, mexa3t_model, misoginia_model]

evo = EvoMSA(TR=True, B4MSA=False, lang='es', Emo=True, Aggress=True, stacked_method=stacked_method, models= mm)

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\single_models\\mexa3t2018_aggress_Es.evomsa'

In [47]:
evo.fit(X_train.squeeze(), y_train.squeeze())
pred = evo.predict(X_test.squeeze())

100%|██████████| 3/3 [03:29<00:00, 69.74s/it]

100%|██████████| 3/3 [03:21<00:00, 67.22s/it]

100%|██████████| 3/3 [03:15<00:00, 65.12s/it]

100%|██████████| 3/3 [01:10<00:00, 23.41s/it]

100%|██████████| 3/3 [01:06<00:00, 22.33s/it]

100%|██████████| 3/3 [01:02<00:00, 20.67s/it]

100%|██████████| 6/6 [00:00<00:00, 47.46it/s]


In [48]:
print(metrics.f1_score(y_test, pred, average="macro"))

0.8363469487658519


In [86]:
# crear modelo pre-entrenado

## Se escala ?

save_model(evo, '..\\..\\models\\haha2021_Es.evomsa') 

In [ ]:
# probarlo
_ = load_model('..\\..\\models\\haha2021_Es.evomsa') 
pred = _.predict(X_test.squeeze())
print(metrics.f1_score(y_test, pred, average="macro"))